In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import random

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# Load cellids into a list

cell_list = [648518346349527319, 648518346349530724, 648518346349537426] # 648518346349538286] #[648518346349538440, 648518346349524070, 648518346349490239, 648518346349525715, 648518346349498574, 648518346349536799]
len(cell_list)

3

In [4]:
# create vasculature list

celltypes = pd.read_csv("data/220309_cell_type_classification.csv", index_col=[0])
vasc = celltypes[celltypes.cell_type == 'vasculature']
vasc_list = vasc.cell_segid.to_list()
len(vasc_list)

72

In [5]:
# Iterate over the cell_list and print the corresponding cell_subtype
for i, cell_id in enumerate(cell_list):
    # Query the cell_subtype for the given cell_id
    cell_subtype = celltypes.loc[celltypes['cell_segid'] == cell_id, 'cell_subtype'].values
    if len(cell_subtype) > 0:
        print(f"index {i}: {cell_subtype[0]}")
    else:
        print(f"index {i}: ID {cell_id} not found in celltypes dataframe")


index 0: astrocyte
index 1: microglia
index 2: pyramidal cell


In [6]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito.head()

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000


In [7]:
mito_list = mito[mito['cellid'].isin(cell_list)].mito_id.values.tolist()
len(mito_list)

1513

In [8]:
mito_list_dictionary_by_cell = {
    f'cell_{uid}': mito[mito.cellid == uid].mito_id.tolist()
    for uid in cell_list
}

# print(mito_list_dictionary_by_cell)

In [9]:
type(mito_list_dictionary_by_cell)

dict

In [10]:
mesh_dir = 'data/neuron_meshes_v185/dec/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/dec/'
mito_source = "precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [11]:
# generate trimesh objects of vascular and cell cells and place in a dictionary

def update_mesh_dictionary(cell_list, mesh_dictionary, prefix, mesh_dir):
    for cellid in cell_list:
        mesh_file = os.path.join(mesh_dir, str(cellid) + '.ply')
        mesh_dictionary[prefix + str(cellid)] = trimesh.load_mesh(mesh_file)

# Initialize the mesh dictionary
mesh_dictionary = {}

# Update the dictionary with cell_list
update_mesh_dictionary(cell_list, mesh_dictionary, 'cell_', mesh_dir)

# Update the dictionary with vasc_list
update_mesh_dictionary(vasc_list, mesh_dictionary, 'vasc_', mesh_dir)


In [12]:
len(mesh_dictionary)

20

In [13]:
# mito_list[:10]

In [14]:
# Create the nested dictionary of decimated mito meshes

trimesh_dictionary_by_cell = {}

for cell_key, mito_list in mito_list_dictionary_by_cell.items():
    # Initialize the second-level dictionary
    nested_dict = {}
    
    for mitoid in mito_list:
        # Construct the file path for the ply file
        ply_file = os.path.join(mito_mesh_dir, str(mitoid) + '.ply')
        
        # Debug: Print the file path to check correctness
        # print(f'Processing file: {ply_file}')
        
        # Check if the file exists and is accessible
        if not os.path.isfile(ply_file):
            print(f'Error: File does not exist - {ply_file}')
            continue
        
        try:
            # Load the ply file as a trimesh object
            mito_mesh = trimesh.load_mesh(ply_file)
            # Define the key for the second-level dictionary
            nested_key = f'trimesh_mito_{mitoid}'
            # Store the trimesh object in the nested dictionary
            nested_dict[nested_key] = mito_mesh
        except Exception as e:
            print(f'Error processing file {ply_file}: {e}')
    
    # Store the nested dictionary in the first-level dictionary
    trimesh_dictionary_by_cell[cell_key] = nested_dict

# Now, trimesh_dictionary_by_cell contains the organized trimesh objects


In [15]:
len(trimesh_dictionary_by_cell)

3

In [16]:
# Display the first 10 items in the nested dictionary

item_count = 0
for cell_key, nested_dict in trimesh_dictionary_by_cell.items():
    print(f'{cell_key}:')
    for nested_key, mito_mesh in nested_dict.items():
        print(f'  {nested_key}: {mito_mesh}')
        item_count += 1
        if item_count >= 10:
            break
    if item_count >= 10:
        break


cell_648518346349527319:
  trimesh_mito_964170: <trimesh.Trimesh(vertices.shape=(30, 3), faces.shape=(55, 3))>
  trimesh_mito_1083380: <trimesh.Trimesh(vertices.shape=(5638, 3), faces.shape=(11383, 3))>
  trimesh_mito_1083414: <trimesh.Trimesh(vertices.shape=(1828, 3), faces.shape=(3774, 3))>
  trimesh_mito_1209147: <trimesh.Trimesh(vertices.shape=(95, 3), faces.shape=(190, 3))>
  trimesh_mito_1083417: <trimesh.Trimesh(vertices.shape=(11, 3), faces.shape=(10, 3))>
  trimesh_mito_1074216: <trimesh.Trimesh(vertices.shape=(71, 3), faces.shape=(138, 3))>
  trimesh_mito_1304771: <trimesh.Trimesh(vertices.shape=(5, 3), faces.shape=(6, 3))>
  trimesh_mito_966479: <trimesh.Trimesh(vertices.shape=(344, 3), faces.shape=(692, 3))>
  trimesh_mito_1664351: <trimesh.Trimesh(vertices.shape=(106, 3), faces.shape=(213, 3))>
  trimesh_mito_1194458: <trimesh.Trimesh(vertices.shape=(9, 3), faces.shape=(9, 3))>


Render all cells with the same settings and pull the viewpoint centroid from an index in cell_list (default index [0])

In [17]:
# Render all cells with the same settings and pull the viewpoint centroid from an index in cell_list (default index [0])

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.2

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

mito_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )

locals().update(mesh_actor)

# Creating a camera object and defining the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000023207E0F360) at 0x0000023216A91408>

Render all cells with the same settings and calculate a global mean of the viewpoint centroid

In [18]:
# Render all cells with the same settings and calculate a global mean of the viewpoint centroid

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.25

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

mito_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000023205EC6120) at 0x0000023216A91D68>

Use different opacity settings for cell in the index 0 position of the list (highlight)

In [19]:
# use different opacity settings for cell in the index 0 position of the list (highlight)

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.05
highlight_opac = 0.5  # Opacity for the first cell in the list

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.01

mito_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Process the first cell separately with highlight_opac
if cell_list:
    first_cellid = cell_list[0]
    first_cell_key = 'cell_' + str(first_cellid)
    if first_cell_key in mesh_dictionary:
        mesh_actor[first_cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[first_cell_key], 
            opacity=highlight_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[first_cell_key].centroid)

# Process the remaining cells with cell_opac
for cellid in cell_list[1:]:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Process the vasc_list as before
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000023205EC4E30) at 0x0000023216AB9768>

Render vasculature and all cells, using random cell colors and a range of cell opacities

In [20]:
# Render vasculature and all cells, using random cell colors and a range of cell opacities

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.025

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        # Randomize cell_color and cell_opac for each mesh
        cell_color = np.random.random(size=3)  # Random color
        cell_opac = np.random.uniform(0.1, 0.25)  # Random opacity between 0.005 and 0.05
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000023205EC34F0) at 0x0000023216AB9468>

Render vasculature, a highlighted cell from index 0, and remaining cells, using random cell colors and a range of cell opacities

In [21]:
# Render vasculature, a highlighted cell from index 0, and remaining cells, using random cell colors and a range of cell opacities

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.025

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Process the first cell separately with a random color and highlight_opac
highlight_opac = 0.025  # Opacity for the highlighted cell

if cell_list:
    first_cellid = cell_list[0]
    first_cell_key = 'cell_' + str(first_cellid)
    if first_cell_key in mesh_dictionary:
        # Randomize the color for the highlighted cell
        highlight_color = np.random.random(size=3)  # Random color
        mesh_actor[first_cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[first_cell_key], 
            opacity=highlight_opac, 
            color=highlight_color
        )
        centroids.append(mesh_dictionary[first_cell_key].centroid)

# Process the remaining cells with random colors and opacities
for cellid in cell_list[1:]:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        # Randomize cell_color and cell_opac for each mesh
        cell_color = np.random.random(size=3)  # Random color
        cell_opac = np.random.uniform(0.2, 0.5)  # Random opacity between a min and max range
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Process the vasc_list as before
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000023205EC6770) at 0x0000023216AB92E8>

# render decimated mitochondria meshes

Render vasc with all cell mitochondria

In [22]:
# Render vasc with all cell mitochondria

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.05

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.025

mito_opac = 0.5

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# uncomment to render cell bodies
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Creating a new dictionary to hold the mito mesh actors
mito_mesh_actor = {}

# Randomly assign colors to each group of cell meshes and add them to the actors
for cell_key, nested_dict in trimesh_dictionary_by_cell.items():
    # Generate a random color for this group of cell meshes
    cell_random_color = [random.random(), random.random(), random.random()]
    
    for mitoid, mito_mesh in nested_dict.items():
        # Define the key for the mesh actor
        actor_key = f'{cell_key}_trimesh_mito_{mitoid}'
        
        # Check if the loaded object is a Scene
        if isinstance(mito_mesh, trimesh.Scene):
            # Extract geometries from the Scene
            for name, geom in mito_mesh.geometry.items():
                mito_mesh = geom
                # Create the mesh actor with the random color and opacity
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=mito_opac, 
                    color=cell_random_color
                )
                centroids.append(mito_mesh.centroid)
        else:
            # Create the mesh actor with the random color and opacity
            mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                mito_mesh, 
                opacity=mito_opac, 
                color=cell_random_color
            )
            centroids.append(mito_mesh.centroid)

# Merge the mito mesh actors into the main mesh_actor dictionary
mesh_actor.update(mito_mesh_actor)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=300)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000002320F8BEED0) at 0x000002327B9F73A8>

Render all mito in all cells with vasc

In [23]:
# render all mito in all cells with vasc

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.2
highlight_opac = 0.025  # Opacity for the highlighted cell

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0

mito_opac = 0.5

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Process the first cell separately with highlight_opac
if cell_list:
    first_cellid = cell_list[0]
    first_cell_key = 'cell_' + str(first_cellid)
    if first_cell_key in mesh_dictionary:
        mesh_actor[first_cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[first_cell_key], 
            opacity=highlight_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[first_cell_key].centroid)

# Process the remaining cells with the standard cell_opac
for cellid in cell_list[1:]:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Process the vasculature list
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Creating a new dictionary to hold the mito mesh actors
mito_mesh_actor = {}

# Randomly assign colors to each group of cell meshes and add them to the actors
for cell_key, nested_dict in trimesh_dictionary_by_cell.items():
    # Generate a random color for this group of cell meshes
    cell_random_color = [random.random(), random.random(), random.random()]
    
    for mitoid, mito_mesh in nested_dict.items():
        # Define the key for the mesh actor
        actor_key = f'{cell_key}_trimesh_mito_{mitoid}'
        
        # Check if the loaded object is a Scene
        if isinstance(mito_mesh, trimesh.Scene):
            # Extract geometries from the Scene
            for name, geom in mito_mesh.geometry.items():
                mito_mesh = geom
                # Create the mesh actor with the random color and opacity
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=mito_opac, 
                    color=cell_random_color
                )
                centroids.append(mito_mesh.centroid)
        else:
            # Create the mesh actor with the random color and opacity
            mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                mito_mesh, 
                opacity=mito_opac, 
                color=cell_random_color
            )
            centroids.append(mito_mesh.centroid)

# Merge the mito mesh actors into the main mesh_actor dictionary
mesh_actor.update(mito_mesh_actor)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=300)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000002320DED0930) at 0x00000232231407C8>

Render all mitos in all cells with vasc, using separate cell and mito opacity settings for the cellid in index 0 position

In [24]:
# render all mitos in all cells with vasc, using separate cell and mito opacity settings for the cellid in index 0 position

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.075
highlight_opac = 0.001  # Opacity for the highlighted cell
highlight_mito_opac = 0.05  # Opacity for the mitochondria of the highlighted cell

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.01

mito_opac = 0.65  # Opacity for the mitochondria of the remaining cells

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []
mito_mesh_actor = {}

# Process the first cell separately with highlight_opac and highlight_mito_opac
if cell_list:
    first_cellid = cell_list[0]
    first_cell_key = 'cell_' + str(first_cellid)
    if first_cell_key in mesh_dictionary:
        # Process the cell body with highlight_opac
        mesh_actor[first_cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[first_cell_key], 
            opacity=highlight_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[first_cell_key].centroid)
    
    # Process the mitochondria of the highlighted cell with highlight_mito_opac
    if first_cell_key in trimesh_dictionary_by_cell:
        for mitoid, mito_mesh in trimesh_dictionary_by_cell[first_cell_key].items():
            actor_key = f'{first_cell_key}_trimesh_mito_{mitoid}'
            if isinstance(mito_mesh, trimesh.Scene):
                for name, geom in mito_mesh.geometry.items():
                    mito_mesh = geom
                    mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                        mito_mesh, 
                        opacity=highlight_mito_opac, 
                        color=cell_color  # Use the same cell color for the highlighted mitochondria
                    )
                    centroids.append(mito_mesh.centroid)
            else:
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=highlight_mito_opac, 
                    color=cell_color
                )
                centroids.append(mito_mesh.centroid)

# Process the remaining cells with the standard cell_opac and random mito_opac and colors
for cellid in cell_list[1:]:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)
    
    if cell_key in trimesh_dictionary_by_cell:
        # Generate a random color for each group of mitochondria
        cell_random_color = [random.random(), random.random(), random.random()]
        
        for mitoid, mito_mesh in trimesh_dictionary_by_cell[cell_key].items():
            actor_key = f'{cell_key}_trimesh_mito_{mitoid}'
            if isinstance(mito_mesh, trimesh.Scene):
                for name, geom in mito_mesh.geometry.items():
                    mito_mesh = geom
                    mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                        mito_mesh, 
                        opacity=mito_opac, 
                        color=cell_random_color  # Assign random color to each group of mitochondria
                    )
                    centroids.append(mito_mesh.centroid)
            else:
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=mito_opac, 
                    color=cell_random_color  # Assign random color to each group of mitochondria
                )
                centroids.append(mito_mesh.centroid)

# Process the vasculature list
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Merge the mito mesh actors into the main mesh_actor dictionary
mesh_actor.update(mito_mesh_actor)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=100)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000002320DEE31E0) at 0x0000023223140D08>

Save image to disk

In [25]:
# save image to disk

import datetime

save_dir = 'vtk_images/'
base_name = 'all_cell_mito_with_vasc'
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=250) # 400 default
# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000002320DEE5170) at 0x00000232231404C8>

# add synapses

In [26]:
syn = pd.read_csv('data/221206_pni_synapses_v185.csv')
syn.head()

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
0,2477272,648518346349538285,648518346349536759,1103,280784,261896,40600,70050,65534,1015,70196,65474,1015,70104,65602,1015
1,4538436,648518346342404863,648518346342414574,62,365832,292808,25800,91472,73196,645,91458,73202,645,91396,73216,645
2,1124039,648518346342921567,648518346345323296,62,229144,155952,48080,57250,39028,1202,57286,38988,1202,57312,38966,1203
3,5245264,648518346342796822,648518346349537255,955,398452,208560,50760,99538,52184,1276,99613,52140,1269,99550,52098,1276
4,6928613,648518346342800412,648518346348190542,718,461868,245212,53560,115462,61350,1335,115467,61303,1339,115468,61296,1335


In [27]:
cell_id

648518346349537426

In [28]:
index_num = 2
seg_id = cell_list[index_num]
seg_id

648518346349537426

In [29]:
# filter all the pre- and post- synapses for this cell
cell_post_subgraph = syn.query(f'post_root_id=={seg_id}') #this shows all post-syn sites on this seg_id cell
cell_pre_subgraph = syn.query(f'pre_root_id=={seg_id}') #this one shows all pre-syn sites on the axon of seg_id cell

# make an array of xyz synapse positions
postsyn_xyz=cell_post_subgraph[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values
presyn_xyz=cell_pre_subgraph[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values

In [30]:
print(f"post-synaptic sites:",len(postsyn_xyz))
print(f"pre-synaptic sites:",len(presyn_xyz))

post-synaptic sites: 2109
pre-synaptic sites: 70


In [39]:
# render all mitos in all cells with vasc, using separate cell and mito opacity settings for the cellid in index 0 position
# add synapses from a cell defined in the previous code block

# settings to visualize pre and post synaptic sites (as dots)
pre_color = (0.2, 0.9, 0.2)
post_color = (0.9, 0.2, 0.2)
pre_opac = 1
post_opac = 1
pre_size = 250
post_size = 250
presyn_actor = trimesh_vtk.point_cloud_actor(presyn_xyz, size=pre_size, opacity=pre_opac, color=pre_color)  
postsyn_actor = trimesh_vtk.point_cloud_actor(postsyn_xyz, size=post_size, opacity=post_opac, color=post_color)

# Define colors and opacities
# cell_color = [0.5, 0.5, 0.5]  # Neutral gray # changed to random color in the code below
cell_opac = 0.2 #0.075
highlight_opac = 0.001  # Opacity for the highlighted cell
highlight_mito_opac = 0.075  # Opacity for the mitochondria of the highlighted cell

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.02

mito_opac = 0.65  # Opacity for the mitochondria of the remaining cells

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []
mito_mesh_actor = {}

# Process the first cell separately with highlight_opac and highlight_mito_opac
if cell_list:
    first_cellid = cell_list[0]
    first_cell_key = 'cell_' + str(first_cellid)
    if first_cell_key in mesh_dictionary:
        # Process the cell body with highlight_opac
        mesh_actor[first_cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[first_cell_key], 
            opacity=highlight_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[first_cell_key].centroid)
    
    # Process the mitochondria of the highlighted cell with highlight_mito_opac
    if first_cell_key in trimesh_dictionary_by_cell:
        for mitoid, mito_mesh in trimesh_dictionary_by_cell[first_cell_key].items():
            actor_key = f'{first_cell_key}_trimesh_mito_{mitoid}'
            if isinstance(mito_mesh, trimesh.Scene):
                for name, geom in mito_mesh.geometry.items():
                    mito_mesh = geom
                    mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                        mito_mesh, 
                        opacity=highlight_mito_opac, 
                        color=cell_color  # Use the same cell color for the highlighted mitochondria
                    )
                    centroids.append(mito_mesh.centroid)
            else:
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=highlight_mito_opac, 
                    color=cell_color
                )
                centroids.append(mito_mesh.centroid)

# Process the remaining cells with the standard cell_opac and random mito_opac and colors
for cellid in cell_list[1:]:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=[random.random(), random.random(), random.random()] #cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)
    
    if cell_key in trimesh_dictionary_by_cell:
        # Generate a random color for each group of mitochondria
        cell_random_color = [random.random(), random.random(), random.random()]
        
        for mitoid, mito_mesh in trimesh_dictionary_by_cell[cell_key].items():
            actor_key = f'{cell_key}_trimesh_mito_{mitoid}'
            if isinstance(mito_mesh, trimesh.Scene):
                for name, geom in mito_mesh.geometry.items():
                    mito_mesh = geom
                    mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                        mito_mesh, 
                        opacity=mito_opac, 
                        color=cell_random_color  # Assign random color to each group of mitochondria
                    )
                    centroids.append(mito_mesh.centroid)
            else:
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=mito_opac, 
                    color=cell_random_color  # Assign random color to each group of mitochondria
                )
                centroids.append(mito_mesh.centroid)

# Process the vasculature list
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Merge the mito mesh actors into the main mesh_actor dictionary
mesh_actor.update(mito_mesh_actor)

# add pre and post synaptic sites to the mesh actor
mesh_actor['presyn_actor'] = presyn_actor
mesh_actor['postsyn_actor'] = postsyn_actor

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=100)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000023304B8E4D0) at 0x00000232438BEF48>

Save image to disk

In [41]:
# save image to disk

import datetime

save_dir = 'vtk_images/'
base_name = 'all_cell_mito_with_vasc_syn'
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=100) # 400 default
trimesh_vtk.render_actors(mesh_actor.values(),
                          filename=filename,
                          do_save=True,
                          scale=6,
                          camera=camera
                         )

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000023304B95CC0) at 0x0000023243E86A68>